In [1]:
!pip install apyori
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
from apyori import apriori
from itertools import combinations
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from tempfile import mkdtemp
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=2f5e98a875e2617d56ebe03d0e814728c49fb547fb1809b0f2e575b6f51f3ff2
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [2]:
%cd 'drive/My Drive/Colab Notebooks/ML TOR'
!pwd

/content/drive/My Drive/Colab Notebooks/ML TOR
/content/drive/My Drive/Colab Notebooks/ML TOR


In [3]:
df = pd.read_csv( 'TorontoData.csv')


In [4]:
df.head()

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Long,Lat,ObjectId
0,-8816401.413,5434587.125,701,GO-20141756319,2014/03/24 00:00:00+00,2014/03/24 00:00:00+00,Commercial,1430,100,Assault,2014,March,24,83,Monday,3,2014.0,March,24.0,83.0,Monday,1,Assault,D42,132,Malvern (132),-79.199081,43.800281,1
1,-8837251.743,5413356.630,901,GO-20143006885,2014/09/27 00:00:00+00,2014/09/29 00:00:00+00,Other,2120,200,B&E,2014,September,29,272,Monday,10,2014.0,September,27.0,270.0,Saturday,16,Break and Enter,D52,76,Bay Street Corridor (76),-79.386383,43.662472,2
2,-8862433.492,5422275.764,702,GO-20141756802,2014/03/24 00:00:00+00,2014/03/24 00:00:00+00,Commercial,2120,200,B&E,2014,March,24,83,Monday,7,2014.0,March,24.0,83.0,Monday,6,Break and Enter,D23,1,West Humber-Clairville (1),-79.612595,43.720406,3
3,-8833103.756,5431886.975,703,GO-20141760570,2014/03/24 00:00:00+00,2014/03/24 00:00:00+00,Apartment,2120,200,B&E,2014,March,24,83,Monday,23,2014.0,March,24.0,83.0,Monday,15,Break and Enter,D33,47,Don Valley Village (47),-79.349121,43.782772,4
4,-8845310.751,5413667.148,902,GO-20142004859,2014/05/03 00:00:00+00,2014/05/03 00:00:00+00,Commercial,1610,210,Robbery - Business,2014,May,3,123,Saturday,2,2014.0,May,3.0,123.0,Saturday,2,Robbery,D11,90,Junction Area (90),-79.458778,43.664490,5


#Data Processing

In [5]:
# Columns for the models
col_list = ['occurrenceyear',	'occurrencemonth','occurrenceday','occurrencedayofyear','occurrencedayofweek','occurrencehour','MCI',	'Division',	'Hood_ID','premisetype']

# New dataframe from columns
df2 = df[col_list]
df2 = df2[df2['occurrenceyear'] > 2013]

#Factorize
crime_var = pd.factorize(df2['MCI'])
df2['MCI'] = crime_var[0]
definition_list_MCI = crime_var[1]

premise_var = pd.factorize(df2['premisetype'])
df2['premisetype'] = premise_var[0]
definition_list_premise = premise_var[1] 

year_var = pd.factorize(df2['occurrenceyear'])
df2['occurrenceyear'] = year_var[0]
definition_list_year = year_var[1] 

month_var = pd.factorize(df2['occurrencemonth'])
df2['occurrencemonth'] = month_var[0]
definition_list_month = month_var[1] 

day_var = pd.factorize(df2['occurrenceday'])
df2['occurenceday'] = day_var[0]
definition_list_day = day_var[1] 

dayweek_var = pd.factorize(df2['occurrencedayofweek'])
df2['occurrencedayofweek'] = dayweek_var[0]
definition_list_day = dayweek_var[1] 

division_var = pd.factorize(df2['Division'])
df2['Division'] = division_var[0]
definition_list_division = division_var[1] 

hood_var = pd.factorize(df2['Hood_ID'])
df2['Hood_ID'] = hood_var[0]
definition_list_hood = hood_var[1] 

hour_var = pd.factorize(df2['occurrencehour'])
df2['occurrencehour'] = hour_var[0]
definition_list_hour = hour_var[1] 

dayyear_var = pd.factorize(df2['occurrencedayofyear'])
df2['occurrencedayofyear'] = dayyear_var[0]
definition_list_dayyear = dayyear_var[1]




#Splitting the dataset into training and tests set

In [6]:
#set X and Y:
X = df2.drop(['MCI'],axis=1).values
y = df2['MCI'].values

#split the data into train and test sets for numeric encoded dataset:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

#need to OneHotEncode all the X variables for input into the classification model:
binary_encoder = OneHotEncoder(sparse=False,categories='auto')
encoded_X = binary_encoder.fit_transform(X)
X_train_OH, X_test_OH, y_train_OH, y_test_OH = train_test_split(encoded_X, y, test_size = 0.3, random_state = 21)

#RandomForestClassifier

In [12]:
#Balanced Class Weight doesn't make a big difference for results:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test) 
y_pred


array([1, 0, 4, ..., 4, 0, 2])

In [14]:
print("Accuracy of Random Forest Training: ",classifier.score(X_train, y_train)* 100)
print("Accuracy of Random Forest: ",accuracy_score(y_test, y_pred)* 100)

print(confusion_matrix(y_test, y_pred)) 
print(classification_report(y_test,y_pred, target_names=definition_list_MCI))

Accuracy of Random Forest Training:  98.61484511981298
Accuracy of Random Forest:  63.78017182599209
[[23917  2186   631    53   819]
 [ 5460  5057    53    50   263]
 [ 2964   330  1639    19   388]
 [ 1098   374    34    26   129]
 [ 2766   707   239    29  2100]]
                 precision    recall  f1-score   support

        Assault       0.66      0.87      0.75     27606
Break and Enter       0.58      0.46      0.52     10883
        Robbery       0.63      0.31      0.41      5340
     Theft Over       0.15      0.02      0.03      1661
     Auto Theft       0.57      0.36      0.44      5841

       accuracy                           0.64     51331
      macro avg       0.52      0.40      0.43     51331
   weighted avg       0.61      0.64      0.61     51331



#KNN

In [7]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [8]:
y_pred = classifier.predict(X_test)
y_pred

array([1, 0, 0, ..., 4, 0, 0])

In [9]:
from sklearn.metrics import classification_report, confusion_matrix
print("Accuracy of KNN Training: ",classifier.score(X_train, y_train)* 100)
print("Accuracy of KNN: ",accuracy_score(y_test, y_pred)* 100)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy of KNN Training:  0.7285602961231249
Accuracy of KNN:  0.515672790321638
[[22858  3159  1002   165   422]
 [ 7944  2242   418    76   203]
 [ 3524   774   911    30   101]
 [ 1249   284    64    28    36]
 [ 4040   976   303    91   431]]
              precision    recall  f1-score   support

           0       0.58      0.83      0.68     27606
           1       0.30      0.21      0.24     10883
           2       0.34      0.17      0.23      5340
           3       0.07      0.02      0.03      1661
           4       0.36      0.07      0.12      5841

    accuracy                           0.52     51331
   macro avg       0.33      0.26      0.26     51331
weighted avg       0.45      0.52      0.46     51331



#AdaBoostClassifier

In [10]:
from sklearn.ensemble import AdaBoostClassifier
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=100,learning_rate=0.001)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
print("Accuracy of AdaBoostClassifier Training: ",classifier.score(X_train, y_train) * 100)
print("Accuracy of AdaBoostClassifier: ",accuracy_score(y_test, y_pred) * 100)

Accuracy of AdaBoostClassifier Training:  0.7285602961231249
Accuracy of AdaBoostClassifier:  0.5378036664004208


#Hyperparameter Tunnning

In [20]:
cachedir = mkdtemp()
pipe = Pipeline([('classify', RandomForestClassifier())],
                memory=cachedir)
# Hyperparameter Optimization
tuned_parameters = [
    # KNN Classifier(2,3,4)
    {
        'classify': [KNeighborsClassifier()],
        'classify__n_neighbors': [1,2]
    },
    #RandomForestClassifier
    {
        'classify': [RandomForestClassifier()],
        'classify__n_estimators': [100, 300],
        "classify__criterion":['gini', 'entropy'],
       # "classify__max_depth":[5,8,15,25,30,None],
       # "classify__min_samples_leaf":[1,2,5,10,15,100],
       # "classify__max_leaf_nodes": [2, 5,10]
        
    },
    #AdaBoostClassifier
    {
        'classify': [AdaBoostClassifier()],
        'classify__n_estimators': [100],
       # 'classify__learning_rate': [1.0]
    }
]

# Model Selection using Pipeline and Cross validation
kfolds = KFold(n_splits=5, shuffle=True, random_state=50)
model = GridSearchCV(pipe, tuned_parameters, cv=kfolds,
                     return_train_score=True)
model.fit(X_train, y_train)
results = pd.DataFrame(model.cv_results_)


In [21]:
results.sort_values(by='mean_test_score', ascending=False).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classify,param_classify__n_neighbors,param_classify__criterion,param_classify__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
3,68.370208,0.459713,3.677190,0.046670,"RandomForestClassifier(bootstrap=True, ccp_alp...",NaN,gini,300,{'classify': RandomForestClassifier(bootstrap=...,0.628547,0.625690,0.627541,0.630560,0.628158,0.628099,0.001573,1,0.988181,0.988441,0.988587,0.988043,0.988173,0.988285,0.000199
5,96.192791,0.804271,3.629290,0.020001,"RandomForestClassifier(bootstrap=True, ccp_alp...",NaN,entropy,300,{'classify': RandomForestClassifier(bootstrap=...,0.628515,0.625528,0.627736,0.629911,0.626307,0.627599,0.001560,2,0.988181,0.988441,0.988587,0.988043,0.988173,0.988285,0.000199
2,22.895578,0.101496,1.225155,0.011046,"RandomForestClassifier(bootstrap=True, ccp_alp...",NaN,gini,100,{'classify': RandomForestClassifier(bootstrap=...,0.628060,0.624131,0.626859,0.628385,0.625268,0.626541,0.001627,3,0.988173,0.988441,0.988579,0.988035,0.988173,0.988280,0.000199
4,32.603858,0.193908,1.217216,0.018160,"RandomForestClassifier(bootstrap=True, ccp_alp...",NaN,entropy,100,{'classify': RandomForestClassifier(bootstrap=...,0.626145,0.624034,0.627249,0.627638,0.625917,0.626197,0.001260,4,0.988173,0.988441,0.988579,0.988035,0.988173,0.988280,0.000199
6,9.313792,0.056648,0.678210,0.007494,"AdaBoostClassifier(algorithm='SAMME.R', base_e...",NaN,NaN,100,{'classify': AdaBoostClassifier(algorithm='SAM...,0.558835,0.557861,0.556595,0.557666,0.554127,0.557017,0.001611,5,0.556172,0.559436,0.556984,0.554671,0.557975,0.557048,0.001611


In [25]:
# Best Model
best_estimator_ = model.best_estimator_
best_estimator_

Pipeline(memory='/tmp/tmpu4stvmq4',
         steps=[('classify',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=300, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [26]:
y_pred = pd.DataFrame()
y_pred['predicted'] = best_estimator_.predict(X_test)


In [27]:
print("best_estimator_: ",accuracy_score(y_test, y_pred)* 100)

best_estimator_:  0.6388342327248642
